categorical のハッシュを取って製品を分け、陰性の製品の代表をいくつか取ってそれらとの差分をまとめ、そのminを取る

In [1]:
import pandas as pd
import numpy as np
import hashlib

In [2]:
line = "L3"

In [3]:
categorical = pd.read_hdf("../hdf/train_categorical.hdf")

In [4]:
column = [c for c in categorical.columns if line in c]

In [5]:
categorical = categorical.loc[:, column]

In [6]:
response = pd.read_hdf("../hdf/train_response.hdf")

In [7]:
c_name = line + "_C_md5"
categorical[c_name] = categorical.apply(
    lambda x: int(hashlib.md5(x.values).hexdigest(), 16),
    axis=1).astype(np.float32)

In [8]:
categorical = pd.concat([categorical, response], axis=1)

In [13]:
# 一定数以上存在する hash を集めます
threshold = 1000
c0 = categorical[categorical["Response"] == 0.0][c_name].value_counts()
famous_hashes = c0[c0 > threshold].index.values
famous_hashes

array([  2.25562989e+38,   1.57961962e+38,   2.92848322e+38,
         1.14759364e+38,   9.51016326e+37,   2.78900494e+38,
         1.13224047e+38,   2.68078571e+38,   7.02657004e+37,
         1.96570684e+38,   5.07292879e+37])

In [14]:
famous_ids = [categorical[categorical[c_name] == hash_num].index.values[0]
              for hash_num in famous_hashes]
famous_ids

[7.0, 4.0, 103.0, 28.0, 616.0, 2410.0, 116.0, 70.0, 2050.0, 3846.0, 146.0]

In [15]:
# 差分計算用 DataFrame
calc_L3 = categorical.drop([c_name, "Response"], axis=1)
diff_L3 = pd.DataFrame(index=calc_L3.index, columns=[])

In [16]:
calc_L3.fillna(0.0, inplace=True)
for famous_id in famous_ids:
    df = calc_L3 - calc_L3.loc[famous_id]
    df.where(df == 0.0, 1.0, inplace=True)
    key = line + "_C_diff_" + str(int(famous_id))
    diff_L3[key] = df.T.sum()
    print(famous_id)

7.0
4.0
103.0
28.0
616.0
2410.0
116.0
70.0
2050.0
3846.0
146.0


In [17]:
diff_L3[line + "_C_diff_min"] = diff_L3.T.min()
diff_L3[c_name] = categorical[c_name]

In [19]:
diff_L3.to_hdf("../hdf/train_categorical_diff_L3.hdf", "df", mode="w")